In [1]:
from datetime import datetime, timedelta
from netCDF4 import Dataset


import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import glob

import numpy as np
import math


from netCDF4 import Dataset

import pandas as pd

from metpy.calc import reduce_point_density
import metpy.calc as mpcalc
from metpy.plots import StationPlot, wx_code_to_numeric
from metpy.plots.wx_symbols import sky_cover, current_weather
from metpy.units import units
from metpy.interpolate import interpolate_to_grid, remove_nan_observations
from metpy.io import metar

In [35]:
#set time to plot
start_year  = 2023
start_month = 4
start_day   = 1
start_hour  = 12
start_min   = 0
    
#set plot domain
lat_min = 33
lat_max = 43
lon_min = -123
lon_max = -113
    
dt = datetime(start_year,start_month,start_day,start_hour,start_min)
del_t = datetime.now()-dt
del_t_days = del_t.days + (del_t.seconds/60/60/24)

In [44]:
#get the data
import requests
import gzip
import shutil

if del_t_days>3:
    #download and extract the data
    base_url = 'https://madis-data.cprk.ncep.noaa.gov/madisPublic1/data/archive/'
    url = f'{base_url}{dt:%Y}/{dt:%m}/{dt:%d}/point/raob/netcdf/{dt:%Y%m%d_%H%M}.gz'
    print(url)
    r = requests.get(url,allow_redirects=True)
    open('temp.nc.gz','wb').write(r.content)
    with gzip.open('temp.nc.gz','rb') as f_in:
        with open('temp.nc','wb') as f_out:
            shutil.copyfileobj(f_in,f_out)
    data=Dataset('temp.nc','r')

else:
    base_url = 'https://madis-data.cprk.ncep.noaa.gov/madisPublic1/data/point/raob/netcdf/'
    url = f'{base_url}{dt:%Y%m%d_%H%M}.nc'
    print(url)
    r = requests.get(url,allow_redirects=True)
    open('temp.nc','wb').write(r.content)
    data=Dataset('temp.nc','r')

https://madis-data.cprk.ncep.noaa.gov/madisPublic1/data/point/raob/netcdf/20230401_1200.nc


OSError: [Errno -51] NetCDF: Unknown file format: b'temp.nc'

In [13]:
print(data.variables)

{'nStaticIds': <class 'netCDF4._netCDF4.Variable'>
int32 nStaticIds()
    _FillValue: 0
unlimited dimensions: 
current shape = ()
filling on, 'staticIds': <class 'netCDF4._netCDF4.Variable'>
|S1 staticIds(maxStaticIds, totalIdLen)
unlimited dimensions: 
current shape = (1000, 50)
filling on, default _FillValue of   used, 'lastRecord': <class 'netCDF4._netCDF4.Variable'>
int32 lastRecord(maxStaticIds)
    _FillValue: -1
unlimited dimensions: 
current shape = (1000,)
filling on, 'invTime': <class 'netCDF4._netCDF4.Variable'>
int32 invTime(recNum)
    _FillValue: 0
unlimited dimensions: recNum
current shape = (402,)
filling on, 'prevRecord': <class 'netCDF4._netCDF4.Variable'>
int32 prevRecord(recNum)
    _FillValue: -1
unlimited dimensions: recNum
current shape = (402,)
filling on, 'inventory': <class 'netCDF4._netCDF4.Variable'>
int32 inventory(maxStaticIds)
    _FillValue: 0
unlimited dimensions: 
current shape = (1000,)
filling on, 'globalInventory': <class 'netCDF4._netCDF4.Variable'

In [ ]:
time = [UNIX_TIME + timedelta(seconds=float(t)) for t in data['timeNominal']]
lat = data['latitude'][:]
lat = lat.filled(np.nan)
lon = data['longitude'][:]
lon = lon.filled(np.nan)
temp = data['temperature'][:]
temp = temp.filled(np.nan)
temp[temp<100]=np.nan
temp = (temp-273.15)
dwpt = data['dewpoint'][:]
dwpt = dwpt.filled(np.nan)
dwpt[dwpt<100]=np.nan
dwpt = (dwpt-273.15)
wspd = data['windSpeed'][:]
wspd = wspd.filled(np.nan)
wspd[wspd>250] = np.nan
wdir = data['windDir'][:]
wdir = wdir.filled(np.nan)
wdir[wdir<0] = np.nan
gust = data['windGust'][:]
gust = gust.filled(np.nan)
gust[gust<0] = np.nan
pres = data['seaLevelPress'][:]/100.0
pres = pres.filled(np.nan)
#pres[pres<900] = np.nan
#pres[pres>1100] = np.nan
#print(pres)
name = data['stationName'][:]
name = [i.tostring().decode()[:4] for i in name]
wx = data['presWeather'][:]
pres_wx = np.zeros(len(lat))
wx = [(i.tostring().decode()).rstrip('\x00') for i in wx]
for i in range(len(wx)):
    this_wx = wx[i].split()
    if this_wx == []:
        pres_wx[i] = 0
        continue
    pres_wx[i] = wx_code_to_numeric(this_wx)[0]
pres_wx = pres_wx.astype(int)
#print([wx_code_to_numeric(i) for i in wx])
#sky_cov[sky_cov=="CLR"] = 0.0
#sky_cov[sky_cov=="FEW"] = 0.25 
#print(wx)
    
u,v = mpcalc.wind_components((wspd*units('m/s')).to('knots'),wdir*units.degree)

#u = np.abs(wspd) * np.cos(np.deg2rad(270-wdir))
#u[np.abs(u)>250] = np.nan
#v = np.abs(wspd) * np.sin(np.deg2rad(270-wdir))
#v[np.abs(v)>250] = np.nan

#u=u*units['kts']
#v=v*units['kts']

df = pd.DataFrame(np.transpose(np.array([time,name,name,lon,lat,u,v,temp,dwpt,pres,sky_cvr,pres_wx])),
                      columns = ['time','id','name','lon','lat','u','v','temp','dwpt','pres','wx'])